In [1]:
import pandas as pd
import numpy as np

In [ ]:
data=pd.read_csv('data/Test Set Values.csv')

In [11]:
# remove extraction type (Y), extraction type group (Z), quantity, quantity group

In [18]:
check=data['extraction_type']==data['extraction_type_group']

In [29]:
size(check[check==False])

NameError: name 'size' is not defined